In [1]:
import pandas as pd

In [2]:
data_16S = pd.read_csv('NCOG_21_16S_redo2_asv_count_tax.tsv', sep='\t')
data_18Sv4 = pd.read_csv('NCOG_18sV4_asv_count_tax.tsv', sep='\t')
data_18Sv9 = pd.read_csv('NCOG_18sV9_asv_count_tax_S.tsv', sep='\t')
data_meta = pd.read_csv('NCOG_sample_log_DNA_stvx_meta_2014-2020_mod.tsv', sep='\t')
data_meta['sampleid'] = data_meta['sampleid'].apply(lambda x: 'X' + x)

In [3]:
data_meta['sample_type'].value_counts()

Surf    772
DCM     702
515       7
170       5
Name: sample_type, dtype: int64

In [13]:
data_16S['silva_Taxon']

0        d__Bacteria; p__Proteobacteria; c__Gammaproteo...
1        d__Bacteria; p__Proteobacteria; c__Gammaproteo...
2        d__Bacteria; p__Proteobacteria; c__Alphaproteo...
3        d__Bacteria; p__Proteobacteria; c__Alphaproteo...
4        d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...
                               ...                        
28904                                          d__Bacteria
28905                                          d__Bacteria
28906    d__Bacteria; p__Proteobacteria; c__Alphaproteo...
28907    d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...
28908    d__Bacteria; p__Cyanobacteria; c__Cyanobacteri...
Name: silva_Taxon, Length: 28909, dtype: object

## Sample types:
### 'Surf' IQR: 10-10 m
### 'DCM' IQR: 30-75 m
### 515: 515 m
### 170: 170 m

In [96]:
taxa_16S = data_16S['silva_Taxon'].str.split(';', expand=True).iloc[:, :8]
taxa_16S.columns = ['Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
abundance_cols = pd.Series(data_16S.columns).apply(lambda x: True if x[:3] == 'X20' else False).values
taxa_16S['abundance_values'] = data_16S.loc[:,abundance_cols].sum(axis=1)
taxa_16S = taxa_16S.dropna(subset='Phylum').fillna('___Undetermined')
for col in taxa_16S.columns:
    if col != 'abundance_values':
        taxa_16S[col] = taxa_16S[col].apply(lambda x: x[4:])
top_10_phyla_16S = (
    taxa_16S.groupby('Phylum')['abundance_values'].sum()
    .sort_values(ascending=False).index.to_list()[:10]
)

colors_16S = {
    top_10_phyla_16S[i]: px.colors.qualitative.Plotly[i] for i in range(10)
}
colors_16S['Undetermined'] = '#C0C0C0' # light grey
colors_16S['Other'] = '#71797E' # dark grey


# 18S -----------------------------------------------------------------------
# show these classes separately from their parent phyla
special_18S_classes = [('Dinoflagellata', 'Syndiniales'), ('Ochrophyta', 'Bacillariophyta')]

# 18Sv4 ---------------------------------------------------------------------
taxa_18Sv4 = data_18Sv4['pr2_Taxon'].str.split(';', expand=True).iloc[:, :8]
taxa_18Sv4.columns = ['Kingdom', 'Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
abundance_cols = pd.Series(data_18Sv4.columns).apply(lambda x: True if x[:3] == 'X20' else False).values
taxa_18Sv4['abundance_values'] = data_18Sv4.loc[:,abundance_cols].sum(axis=1)
taxa_18Sv4 = taxa_18Sv4.dropna(subset='Phylum').fillna('Undetermined')
top_8_phyla_18Sv4 = (
    taxa_18Sv4.groupby('Phylum')['abundance_values'].sum()
    .sort_values(ascending=False).index.to_list()[:8]
)

colors_18Sv4 = {
    top_8_phyla_18Sv4[i]: px.colors.qualitative.Plotly[i] for i in range(8)
}
for i, (phylum, special_class) in enumerate(special_18S_classes):
    colors_18Sv4[special_class] = px.colors.qualitative.Plotly[i + 8]
colors_18Sv4['Undetermined'] = '#C0C0C0' # light grey
colors_18Sv4['Other'] = '#71797E' # dark grey

# 18Sv9 ---------------------------------------------------------------------
taxa_18Sv9 = data_18Sv9['pr2_Taxon'].str.split(';', expand=True).iloc[:, :8]
taxa_18Sv9.columns = ['Kingdom', 'Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
abundance_cols = pd.Series(data_18Sv9.columns).apply(lambda x: True if x[:3] == 'X20' else False).values
taxa_18Sv9['abundance_values'] = data_18Sv9.loc[:,abundance_cols].sum(axis=1)
taxa_18Sv9 = taxa_18Sv9.dropna(subset='Phylum').fillna('Undetermined')
top_8_phyla_18Sv9 = (
    taxa_18Sv9.groupby('Phylum')['abundance_values'].sum()
    .sort_values(ascending=False).index.to_list()[:8]
)

colors_18Sv9 = {
    top_8_phyla_18Sv9[i]: px.colors.qualitative.Plotly[i] for i in range(8)
}
for i, (phylum, special_class) in enumerate(special_18S_classes):
    colors_18Sv9[special_class] = px.colors.qualitative.Plotly[i + 8]
colors_18Sv9['Undetermined'] = '#C0C0C0' # light grey
colors_18Sv9['Other'] = '#71797E' # dark grey

colors_18Sv9 = colors_18Sv4 # the top 8 groups should match but I'll leave the code here just in case

# for phylum in ['Dinoflagellata', 'Ochrophyta']:
#     dino_ochro_classes = taxa_18Sv4[taxa_18Sv4['Phylum'] == phylum]['Class']
#     dino_ochro_classes = dino_ochro_classes[dino_ochro_classes != 'Undetermined'].unique()
#     special_classes.extend(dino_ochro_classes.tolist())
# unique_phyla_18Sv4.extend(dino_ochro_children)
# unique_phyla_18Sv4 = [x for x in unique_phyla_18Sv4 if x not in ['Dinoflagellata', 'Ochrophyta']]

In [9]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import plotly.express as px
import os
import sys
from dotenv import load_dotenv
load_dotenv()

px.set_mapbox_access_token(os.getenv('MAPBOX_ACCESS_TOKEN'))

# Load data
data_16S = pd.read_csv('NCOG_21_16S_redo2_asv_count_tax.tsv', sep='\t')
data_18Sv4 = pd.read_csv('NCOG_18sV4_asv_count_tax.tsv', sep='\t')
data_18Sv9 = pd.read_csv('NCOG_18sV9_asv_count_tax_S.tsv', sep='\t')
data_meta = pd.read_csv('NCOG_sample_log_DNA_stvx_meta_2014-2020_mod.tsv', sep='\t')

# Add X before metadata sampleids to match 16S and 18S sample ids
data_meta['sampleid'] = data_meta['sampleid'].apply(lambda x: 'X' + x)

# Lat and Lon center for plots
cal_coast_center = dict(
    lat=np.mean([min(data_meta['Lat_Dec']), max(data_meta['Lat_Dec'])]),
    lon=np.mean([min(data_meta['Lon_Dec']), max(data_meta['Lon_Dec'])])
)

# Environmental variables and sample types
env_var_cols = ['T_degC', 'Salnty', 'O2ml_L', 'PO4ug', 'SiO3ug', 'NO3ug', 'NH3ug', 'ChlorA', 'IntC14', 'NCDepth']
sample_type_vals = data_meta['sample_type'].dropna().unique()

# precompute map figures for different sample types and environmental variables
map_figs = {sample_type: {} for sample_type in data_meta['sample_type'].dropna().unique()}
for sample_type in data_meta['sample_type'].dropna().unique():
    for env_var in env_var_cols:
        meta_subset = data_meta[data_meta['sample_type'] == sample_type]
        meta_subset = meta_subset[['Sta_ID', 'Lat_Dec', 'Lon_Dec', 'sample_type', env_var]].groupby('Sta_ID').agg({
            'Lat_Dec': 'min',
            'Lon_Dec': 'min',
            'sample_type': 'count',
            env_var: 'mean'
        }).rename({'sample_type': 'Number of Samples'}, axis=1).reset_index()
        hover_names = meta_subset['Sta_ID'].apply(lambda x: '<b>Station: </b>' + x)
        subset_fig = px.scatter_mapbox(meta_subset, lat='Lat_Dec', lon='Lon_Dec', center=cal_coast_center,
                                       color=env_var, hover_name=hover_names, hover_data='Number of Samples', #size="num_samples",
                                       color_continuous_scale='viridis', size_max=15, zoom=4.5, mapbox_style='outdoors',
                                       width=600, height=700, custom_data='Sta_ID')
        map_figs[sample_type][env_var] = subset_fig

# empty sunburst figure for if the station has no data with the selected sample type
empty_sunburst_data = {'Phylum': [], 'Class': [], 'Order': []}
for parent in ['Undetermined_1', 'Undetermined_2', 'Undetermined_3']:
    for child1 in ['Undetermined_1', 'Undetermined_2', 'Undetermined_3']:
        for child2 in ['Undetermined_1', 'Undetermined_2', 'Undetermined_3']:
            empty_sunburst_data['Phylum'].append(parent)
            empty_sunburst_data['Class'].append(child1)
            empty_sunburst_data['Order'].append(child2)
empty_sunburst_fig = px.sunburst(empty_sunburst_data, path=['Phylum', 'Class', 'Order'])

# precompute sunburst figures for every station id, sample type, dataset combination
counter = 0
total_computing = len(data_meta['Sta_ID'].dropna().unique()) * len(data_meta['sample_type'].dropna().unique()) * 3
print(f'Pre-computing {total_computing} Sunburst Figures')
sunburst_figs = {station_id: {} for station_id in data_meta['Sta_ID'].dropna().unique()}
taxonomies_all = {station_id: {} for station_id in data_meta['Sta_ID'].dropna().unique()}
for station_id in data_meta['Sta_ID'].dropna().unique():
    for sample_type in data_meta['sample_type'].dropna().unique():
        sunburst_figs[station_id][sample_type] = {}
        taxonomies_all[station_id][sample_type] = {}
        for dataset in ['16S', '18Sv4', '18Sv9']:
            cols_show_in_sunburst = ['Phylum', 'Class', 'Order']
            station_data = data_meta[(data_meta['Sta_ID'] == station_id) & (data_meta['sample_type'] == sample_type)]
            station_samples = station_data['sampleid'].tolist()
            if dataset =='16S':
                taxa_col_names = ['Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']

                # Merge with 16S dataframe to get taxonomy data for the samples
                asv_cols = pd.Series(data_16S.columns).isin(station_samples).values
                if np.sum(asv_cols) == 0: # If there are no samples for the sample_type at the station
                    # set the figure to an empty figure
                    title = f'No {sample_type} samples for station "{station_id}"'
                    fig = px.sunburst(empty_sunburst_data, path=['Phylum', 'Class', 'Order'], title=title)
                    sunburst_figs[station_id][sample_type][dataset] = fig
                    continue
                asv_cols[0] = True
                station_asvs = pd.concat([data_16S.loc[:,asv_cols], data_16S['silva_Taxon']], axis=1)

                # Get relative abundances
                values = station_asvs.drop(['Feature.ID', 'silva_Taxon'], axis=1).fillna(0).sum(axis=1)
                #values = values / values.sum()

                # Count occurrences of each taxonomy category
                taxonomies = station_asvs['silva_Taxon'].str.split('; ', expand=True)
                taxonomies.columns = taxa_col_names
                taxonomies = taxonomies.dropna(subset=cols_show_in_sunburst[0]).fillna('___Undetermined')[cols_show_in_sunburst]

                # get rid of the silva d__, p__, etc prefixes
                for col in taxonomies.columns:
                    taxonomies[col] = taxonomies[col].apply(lambda x: x[3:])

                # Get relative abundances
                taxonomies['abundance_values'] = values
                taxonomies = taxonomies[taxonomies['abundance_values'] != 0]

                # set title of plot
                title = '16S Silva Taxonomy, Station "' + station_id + '"'
            elif dataset == '18Sv4':
                taxa_col_names = ['Kingdom', 'Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
                # Merge with 18Sv4 dataframe to get taxonomy data for the samples
                asv_cols = pd.Series(data_18Sv4.columns).isin(station_samples).values
                if np.sum(asv_cols) == 0: # If there are no samples for the sample_type at the station
                    # set the figure to an empty figure
                    title = f'No {sample_type} samples for station "{station_id}"'
                    fig = px.sunburst(empty_sunburst_data, path=['Phylum', 'Class', 'Order'], title=title)
                    sunburst_figs[station_id][sample_type][dataset] = fig
                    continue
                asv_cols[0] = True
                station_asvs = pd.concat([data_18Sv4.loc[:,asv_cols], data_18Sv4['pr2_Taxon']], axis=1)

                # Get relative abundances
                values = station_asvs.drop(['Feature.ID', 'pr2_Taxon'], axis=1).fillna(0).sum(axis=1)
                #values = values / values.sum()

                # Count occurrences of each taxonomy category
                taxonomies = station_asvs['pr2_Taxon'].str.split(';', expand=True)
                taxonomies = taxonomies.iloc[:, :8]
                taxonomies.columns = taxa_col_names
                taxonomies = taxonomies.dropna(subset='Phylum').fillna('Undetermined')[cols_show_in_sunburst]

                # Add relative abundances
                taxonomies['abundance_values'] = values
                taxonomies = taxonomies[taxonomies['abundance_values'] != 0]

                # set title of plot
                title = '18S v4 PR2 Taxonomy, Station "' + station_id + '"'

            elif dataset == '18Sv9':
                taxa_col_names = ['Kingdom', 'Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']

                # Merge with 18Sv9 dataframe to get taxonomy data for the samples
                asv_cols = pd.Series(data_18Sv9.columns).isin(station_samples).values
                if np.sum(asv_cols) == 0: # If there are no samples for the sample_type at the station
                    # set the figure to an empty figure
                    title = f'No {sample_type} samples for station "{station_id}"'
                    fig = px.sunburst(empty_sunburst_data, path=['Phylum', 'Class', 'Order'], title=title)
                    sunburst_figs[station_id][sample_type][dataset] = fig
                    continue
                asv_cols[0] = True
                station_asvs = pd.concat([data_18Sv9.loc[:,asv_cols], data_18Sv9['pr2_Taxon']], axis=1)

                # Get relative abundances
                values = station_asvs.drop(['Feature.ID', 'pr2_Taxon'], axis=1).fillna(0).sum(axis=1)
                #values = values / values.sum()

                # Count occurrences of each taxonomy category
                taxonomies = station_asvs['pr2_Taxon'].str.split(';', expand=True)
                taxonomies = taxonomies.iloc[:, :8]
                taxonomies.columns = taxa_col_names
                taxonomies = taxonomies.dropna(subset='Phylum').fillna('Undetermined')[cols_show_in_sunburst]

                # Add relative abundances
                taxonomies['abundance_values'] = values
                taxonomies = taxonomies[taxonomies['abundance_values'] != 0]

                # set title of plot
                title = '18S v9 PR2 Taxonomy, Station "' + station_id + '"'
            taxonomies['relative_abundance'] = taxonomies['abundance_values'] / taxonomies['abundance_values'].sum()
            taxonomies['relative_abundance'] = (taxonomies['relative_abundance'] * 100).round(2)
            fig = px.sunburst(taxonomies, path=['Phylum', 'Class', 'Order'], values='relative_abundance',
                              title=title)
            taxonomies_all[station_id][sample_type][dataset] = taxonomies
            sunburst_figs[station_id][sample_type][dataset] = fig
        counter += 3
        print('Finished: {0:.2f}%'.format(100 * counter / total_computing), end='\r')
print()


Pre-computing 1092 Sunburst Figures
Finished: 100.00%


# Create stacked bar 

In [190]:
counter = 0
print(f'Pre-computing {total_computing} Stacked Bar Figures')
stacked_bar_figs = {station_id: {} for station_id in data_meta['Sta_ID'].dropna().unique()}
for station_id in data_meta['Sta_ID'].dropna().unique():
    for sample_type in data_meta['sample_type'].dropna().unique():
        stacked_bar_figs[station_id][sample_type] = {}
        for dataset in ['16S', '18Sv4', '18Sv9']:
            station_data = data_meta[(data_meta['Sta_ID'] == station_id) & (data_meta['sample_type'] == sample_type)]
            station_samples = station_data['sampleid'].tolist()
            if dataset == '16S':
                taxa_col_names = ['Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
                asv_cols = pd.Series(data_16S.columns).isin(station_samples).values
                if np.sum(asv_cols) == 0:
                    empty_fig = go.Figure(
                        data=[
                            go.Bar(name='Group 1', x=['No Samples'], y=[1], offsetgroup=1),
                            go.Bar(name='Group 2', x=['No Samples'], y=[1], offsetgroup=1, base=[1])
                        ],
                        layout=go.Layout(
                            title=f'Station {station_id} phyla distribution per sample',
                            yaxis_title='Relative Abundance', height=600, width=600
                        )
                    )
                    stacked_bar_figs[station_id][sample_type][dataset] = empty_fig
                    continue

                asv_cols[0] = True
                station_asvs = pd.concat([data_16S.loc[:,asv_cols], data_16S['silva_Taxon']], axis=1)

                # Get relative abundances
                values = station_asvs.drop(['Feature.ID', 'silva_Taxon'], axis=1).fillna(0)
                values = values / values.sum()

                # Count occurrences of each taxonomy category
                taxonomies = station_asvs['silva_Taxon'].str.split('; ', expand=True)
                taxonomies.columns = taxa_col_names
                taxonomies = taxonomies.fillna('___Undetermined')[['Phylum', 'Class']]

                # get rid of the silva d__, p__, etc prefixes
                for col in taxonomies.columns:
                    taxonomies[col] = taxonomies[col].apply(lambda x: x[3:])

                # Get relative abundances of top phyla / special classes
                taxonomies = pd.concat([taxonomies, values], axis=1)
                phyla_counts = taxonomies.groupby('Phylum').sum()
                for phylum in top_10_phyla_16S:
                    if phylum not in phyla_counts.index:
                        phyla_counts.loc[phylum] = 0
                top_phyla_counts = phyla_counts.loc[top_10_phyla_16S]
                other_phyla_counts = phyla_counts[(
                    (phyla_counts.index != 'Undetermined') & (~phyla_counts.index.isin(top_10_phyla_16S))
                )].sum()
                top_phyla_counts.loc['Other'] = other_phyla_counts
                top_phyla_counts.loc['Undetermined'] = phyla_counts.loc['Undetermined']

                # Create stacked bar figure
                base = top_phyla_counts.loc[top_phyla_counts.index[0]].copy()
                stacked_bar_labels = top_phyla_counts.columns
                fig_data = [go.Bar(
                    name=top_phyla_counts.index[0],
                    x=stacked_bar_labels,
                    y=base,
                    customdata=base.copy(),
                    marker=go.bar.Marker(color=colors_16S[top_phyla_counts.index[0]]),
                    hovertemplate='Rel. Abundance:%{customdata:.3f}',
                    offsetgroup=1,
                )]
                for phyla in top_phyla_counts.index[1:]:
                    vals = top_phyla_counts.loc[phyla]
                    fig_data.append(go.Bar(
                        name=phyla,
                        x=stacked_bar_labels,
                        y=vals,
                        marker=go.bar.Marker(color=colors_16S[phyla]),
                        customdata=vals.copy(),
                        hovertemplate='Rel. Abundance:%{customdata:.3f}',
                        offsetgroup=1,
                        base=base
                    ))
                    base += vals
                stacked_bar_fig = go.Figure(
                    data=fig_data,
                    layout=go.Layout(
                        title=f'Station {station_id} phyla distribution per sample',
                        yaxis_title='Relative abundance'
                    )
                )
                stacked_bar_figs[station_id][sample_type][dataset] =  stacked_bar_fig

            if dataset == '18Sv4':
                taxa_col_names = ['Kingdom', 'Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
                asv_cols = pd.Series(data_18Sv4.columns).isin(station_samples).values
                if np.sum(asv_cols) == 0:
                    empty_fig = go.Figure(
                        data=[
                            go.Bar(name='Group 1', x=['No Samples'], y=[1], offsetgroup=1),
                            go.Bar(name='Group 2', x=['No Samples'], y=[1], offsetgroup=1, base=[1])
                        ],
                        layout=go.Layout(
                            title=f'Station {station_id} phyla distribution per sample',
                            yaxis_title='Relative Abundance', height=600, width=600
                        )
                    )
                    stacked_bar_figs[station_id][sample_type][dataset] = empty_fig
                    continue
                
                asv_cols[0] = True
                station_asvs = pd.concat([data_18Sv4.loc[:,asv_cols], data_18Sv4['pr2_Taxon']], axis=1)

                # Get relative abundances
                values = station_asvs.drop(['Feature.ID', 'pr2_Taxon'], axis=1).fillna(0)
                values = values / values.sum()

                # Count occurrences of each taxonomy category
                taxonomies = station_asvs['pr2_Taxon'].str.split(';', expand=True).iloc[:,:8]
                taxonomies.columns = taxa_col_names
                taxonomies = taxonomies.fillna('Undetermined')[['Phylum', 'Class']]

                # Get relative abundances of top phyla / special classes
                taxonomies = pd.concat([taxonomies, values], axis=1)
                phyla_counts = taxonomies.groupby('Phylum').sum()
                class_counts = taxonomies.groupby('Class').sum()
                if 'Syndiniales' not in class_counts.index:
                    class_counts.loc['Syndiniales'] = 0
                if 'Bacillariophyta' not in class_counts.index:
                    class_counts.loc['Bacillariophyta'] = 0
                for phylum in top_8_phyla_18Sv4 + ['Dinoflagellata', 'Ochrophyta']:
                    if not phylum in phyla_counts.index:
                        phyla_counts.loc[phylum] = 0
                phyla_counts.loc['Dinoflagellata'] -= class_counts.loc['Syndiniales']
                phyla_counts.loc['Ochrophyta'] -= class_counts.loc['Bacillariophyta']
                
                # Top 10 phyla + special classes
                top_phyla_counts = pd.concat([
                    phyla_counts.loc[top_8_phyla_18Sv4],
                    class_counts.loc[['Syndiniales', 'Bacillariophyta']]
                ])
                other_phyla_counts = phyla_counts[(
                    (phyla_counts.index != 'Undetermined') & (~phyla_counts.index.isin(top_10_phyla_16S))
                )].sum()
                top_phyla_counts.loc['Other'] = other_phyla_counts
                top_phyla_counts.loc['Undetermined'] = phyla_counts.loc['Undetermined']

                # Create stacked bar figure
                base = top_phyla_counts.loc[top_phyla_counts.index[0]].copy()
                stacked_bar_labels = top_phyla_counts.columns
                fig_data = [go.Bar(
                    name=top_phyla_counts.index[0],
                    x=stacked_bar_labels,
                    y=base,
                    customdata=base.copy(),
                    marker=go.bar.Marker(color=colors_18S[top_phyla_counts.index[0]]),
                    hovertemplate='Rel. Abundance:%{customdata:.3f}',
                    offsetgroup=1,
                )]
                for phyla in top_phyla_counts.index[1:]:
                    vals = top_phyla_counts.loc[phyla]
                    fig_data.append(go.Bar(
                        name=phyla,
                        x=stacked_bar_labels,
                        y=vals,
                        marker=go.bar.Marker(color=colors_18S[phyla]),
                        customdata=vals.copy(),
                        hovertemplate='Rel. Abundance:%{customdata:.3f}',
                        offsetgroup=1,
                        base=base
                    ))
                    base += vals
                stacked_bar_fig = go.Figure(
                    data=fig_data,
                    layout=go.Layout(
                        title=f'Station {station_id} phyla distribution per sample',
                        yaxis_title='Relative abundance'
                    )
                )
                stacked_bar_figs[station_id][sample_type][dataset] =  stacked_bar_fig
            if dataset == '18Sv9':
                taxa_col_names = ['Kingdom', 'Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
                asv_cols = pd.Series(data_18Sv9.columns).isin(station_samples).values
                if np.sum(asv_cols) == 0:
                    empty_fig = go.Figure(
                        data=[
                            go.Bar(name='Group 1', x=['No Samples'], y=[1], offsetgroup=1),
                            go.Bar(name='Group 2', x=['No Samples'], y=[1], offsetgroup=1, base=[1])
                        ],
                        layout=go.Layout(
                            title=f'Station {station_id} phyla distribution per sample',
                            yaxis_title='Relative Abundance', height=600, width=600
                        )
                    )
                    stacked_bar_figs[station_id][sample_type][dataset] = empty_fig
                    continue
                
                asv_cols[0] = True
                station_asvs = pd.concat([data_18Sv9.loc[:,asv_cols], data_18Sv9['pr2_Taxon']], axis=1)

                # Get relative abundances
                values = station_asvs.drop(['Feature.ID', 'pr2_Taxon'], axis=1).fillna(0)
                values = values / values.sum()

                # Count occurrences of each taxonomy category
                taxonomies = station_asvs['pr2_Taxon'].str.split(';', expand=True).iloc[:,:8]
                taxonomies.columns = taxa_col_names
                taxonomies = taxonomies.fillna('Undetermined')[['Phylum', 'Class']]

                # Get relative abundances of top phyla / special classes
                taxonomies = pd.concat([taxonomies, values], axis=1)
                phyla_counts = taxonomies.groupby('Phylum').sum()
                class_counts = taxonomies.groupby('Class').sum()
                if 'Syndiniales' not in class_counts.index:
                    class_counts.loc['Syndiniales'] = 0
                if 'Bacillariophyta' not in class_counts.index:
                    class_counts.loc['Bacillariophyta'] = 0
                for phylum in top_8_phyla_18Sv9 + ['Dinoflagellata', 'Ochrophyta']:
                    if phylum not in phyla_counts.index:
                        phyla_counts.loc[phylum] = 0
                phyla_counts.loc['Dinoflagellata'] -= class_counts.loc['Syndiniales']
                phyla_counts.loc['Ochrophyta'] -= class_counts.loc['Bacillariophyta']
                
                # Top 10 phyla + special classes
                top_phyla_counts = pd.concat([
                    phyla_counts.loc[top_8_phyla_18Sv9],
                    class_counts.loc[['Syndiniales', 'Bacillariophyta']]
                ])
                other_phyla_counts = phyla_counts[(
                    (phyla_counts.index != 'Undetermined') & (~phyla_counts.index.isin(top_10_phyla_16S))
                )].sum()
                top_phyla_counts.loc['Other'] = other_phyla_counts
                top_phyla_counts.loc['Undetermined'] = phyla_counts.loc['Undetermined']

                # Create stacked bar figure
                base = top_phyla_counts.loc[top_phyla_counts.index[0]].copy()
                stacked_bar_labels = top_phyla_counts.columns
                fig_data = [go.Bar(
                    name=top_phyla_counts.index[0],
                    x=stacked_bar_labels,
                    y=base,
                    customdata=base.copy(),
                    marker=go.bar.Marker(color=colors_18S[top_phyla_counts.index[0]]),
                    hovertemplate='Rel. Abundance:%{customdata:.3f}',
                    offsetgroup=1,
                )]
                for phyla in top_phyla_counts.index[1:]:
                    vals = top_phyla_counts.loc[phyla]
                    fig_data.append(go.Bar(
                        name=phyla,
                        x=stacked_bar_labels,
                        y=vals,
                        marker=go.bar.Marker(color=colors_18S[phyla]),
                        customdata=vals.copy(),
                        hovertemplate='Rel. Abundance:%{customdata:.3f}',
                        offsetgroup=1,
                        base=base
                    ))
                    base += vals
                stacked_bar_fig = go.Figure(
                    data=fig_data,
                    layout=go.Layout(
                        title=f'Station {station_id} phyla distribution per sample',
                        yaxis_title='Relative abundance'
                    )
                )
                stacked_bar_figs[station_id][sample_type][dataset] =  stacked_bar_fig
        counter += 3
        print('Finished: {0:.2f}%'.format(100 * counter / total_computing), end='\r')

Pre-computing 1092 Stacked Bar Figures


In [194]:
dataset = '16S'
station_id = '093.3 030.0'
sample_type = 'Surf'
stacked_bar_figs[station_id][sample_type][dataset]

In [182]:
dataset = '16S'
station_id = '093.3 030.0'
sample_type = 'Surf'
station_data = data_meta[(data_meta['Sta_ID'] == station_id) & (data_meta['sample_type'] == sample_type)]
station_samples = station_data['sampleid'].tolist()
if dataset == '16S':
    taxa_col_names = ['Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
    asv_cols = pd.Series(data_16S.columns).isin(station_samples).values
    if np.sum(asv_cols) == 0:
        empty_fig = go.Figure(
            data=[
                go.Bar(name='Group 1', x=['No Samples'], y=[1], offsetgroup=1),
                go.Bar(name='Group 2', x=['No Samples'], y=[1], offsetgroup=1, base=[1])
            ],
            layout=go.Layout(
                title=f'Station {station_id} phyla distribution per sample',
                yaxis_title='Relative Abundance', height=600, width=600
            )
        )
        stacked_bar_figs[station_id][sample_type][dataset] = empty_fig

    asv_cols[0] = True
    station_asvs = pd.concat([data_16S.loc[:,asv_cols], data_16S['silva_Taxon']], axis=1)

    # Get relative abundances
    values = station_asvs.drop(['Feature.ID', 'silva_Taxon'], axis=1).fillna(0)
    values = values / values.sum()

    # Count occurrences of each taxonomy category
    taxonomies = station_asvs['silva_Taxon'].str.split('; ', expand=True)
    taxonomies.columns = taxa_col_names
    taxonomies = taxonomies.fillna('___Undetermined')[['Phylum', 'Class']]

    # get rid of the silva d__, p__, etc prefixes
    for col in taxonomies.columns:
        taxonomies[col] = taxonomies[col].apply(lambda x: x[3:])

    # Get relative abundances of top phyla / special classes
    taxonomies = pd.concat([taxonomies, values], axis=1)
    phyla_counts = taxonomies.groupby('Phylum').sum()
    for phylum in top_10_phyla_16S:
        if phylum not in phyla_counts.index:
            phyla_counts.loc[phylum] = 0
    top_phyla_counts = phyla_counts.loc[top_10_phyla_16S]
    other_phyla_counts = phyla_counts[(
        (phyla_counts.index != 'Undetermined') & (~phyla_counts.index.isin(top_10_phyla_16S))
    )].sum()
    top_phyla_counts.loc['Other'] = other_phyla_counts
    top_phyla_counts.loc['Undetermined'] = phyla_counts.loc['Undetermined']

    # Create stacked bar figure
    base = top_phyla_counts.loc[top_phyla_counts.index[0]].copy()
    stacked_bar_labels = top_phyla_counts.columns
    fig_data = [go.Bar(
        name=top_phyla_counts.index[0],
        x=stacked_bar_labels,
        y=base,
        customdata=base.copy(),
        marker=go.bar.Marker(color=colors_16S[top_phyla_counts.index[0]]),
        hovertemplate='Rel. Abundance:%{customdata:.3f}',
        offsetgroup=1,
    )]
    for phyla in top_phyla_counts.index[1:]:
        vals = top_phyla_counts.loc[phyla]
        fig_data.append(go.Bar(
            name=phyla,
            x=stacked_bar_labels,
            y=vals,
            marker=go.bar.Marker(color=colors_16S[phyla]),
            customdata=vals.copy(),
            hovertemplate='Rel. Abundance:%{customdata:.3f}',
            offsetgroup=1,
            base=base
        ))
        base += vals
    stacked_bar_fig = go.Figure(
        data=fig_data,
        layout=go.Layout(
            title=f'Station {station_id} phyla distribution per sample',
            yaxis_title='Relative abundance'
        )
    )
    stacked_bar_figs[station_id][sample_type][dataset] =  stacked_bar_fig
stacked_bar_figs[station_id][sample_type][dataset].show()

In [195]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Container([
    html.H1("Marine microbial life at California coastal stations", className="mt-4 mb-4"),
    dbc.Row([
        html.Div([
            html.Label('Sample Type:'),
            dcc.Dropdown(sample_type_vals, sample_type_vals[0], id='sample-type-dropdown')
        ])
    ]),
    dbc.Row([
        dbc.Col([
            html.Div([
                html.Label('Environmental Variable (Color):'),
                dcc.Dropdown(env_var_cols, 'NCDepth', id='env-var-dropdown')
            ], className="env-var-dropdown"),
            html.Div([
                dcc.Graph(id='map-graph')
            ], className="map-graph"),
        ], width=5),
        dbc.Col([
            html.Div([
                html.Label('Dataset:'),
                dcc.Dropdown(['16S', '18Sv4', '18Sv9'], '16S', id='dataset-dropdown')
            ]),
            html.Div([
                dcc.Graph(id='sunburst-graph')
            ]),
            html.Div([
                dcc.Graph(id='stacked-bar-graph')
            ])
        ], width=5)
    ])
], fluid=True)


# Map graph dropdown callback
@app.callback(
    Output('map-graph', 'figure'),
    [Input('sample-type-dropdown', 'value'),
     Input('env-var-dropdown', 'value')]
)
def update_map(dropdown_sample_type, dropdown_env_var):
    return map_figs[dropdown_sample_type][dropdown_env_var]

# Map graph click data callback for sunburst
@app.callback(
    Output('sunburst-graph', 'figure'),
    [Input('sample-type-dropdown', 'value'),
     Input('map-graph', 'clickData'),
     Input('dataset-dropdown', 'value')]
)
def update_sunburst(dropdown_sample_type, click_data, dropdown_dataset):
    if click_data is None:
        station_id = data_meta['Sta_ID'].iloc[0]
    # Get station ID from hover data
    else:
        if 'customdata' not in click_data['points'][0]:
            station_id = station_id = data_meta['Sta_ID'].iloc[0]
        else:
            station_id = click_data['points'][0]['customdata'][0]
    return sunburst_figs[station_id][dropdown_sample_type][dropdown_dataset]

# Stacked bar callback
@app.callback(
    Output('stacked-bar-graph', 'figure'),
    [Input('map-graph', 'clickData'),
     Input('dataset-dropdown', 'value')]
)
def update_stacked_bar(click_data, dropdown_dataset):
    if click_data is None:
        station_id = data_meta['Sta_ID'].iloc[0]
    # Get station ID from hover data
    else:
        if 'customdata' not in click_data['points'][0]:
            station_id = station_id = data_meta['Sta_ID'].iloc[0]
        else:
            station_id = click_data['points'][0]['customdata'][0]
    return stacked_bar_figs[station_id][dropdown_sample_type][dropdown_dataset]


app.run_server(debug=True)


In [ ]:
group_colors = {
    
}
for group in groups_above_threshold:
stacked_bar_fig = go.Figure(
    data=[
        go.Bar(
            name="Model 1",
            x=data["labels"],
            y=data["model_1"],
            offsetgroup=1,
        ),
        go.Bar(
            name="Model 2",
            x=data["labels"],
            y=data["model_2"],
            offsetgroup=1,
            base=data["model_1"], 
        )
    ],
    layout=go.Layout(
        title="Issue Types - Original and Models",
        yaxis_title="Number of Issues"
    )
)
fig3.show()

In [ ]:
map_figs['DCM']['T_degC'].update_layout(
    autosize=True,
    margin=dict(l=10,r=10,b=10,t=10,pad=3),
    paper_bgcolor="LightSteelBlue",
)
map_figs['DCM']['T_degC'].show()